In [10]:
import numpy as np
from calculate_n import *
from calculate_n import _get_z

In [20]:
help(cal_n)
help(cal_ci)
help(cal_n_from_t_test)
help(cal_ratio_for_comparing_two_means)
help(t_test)
help(create_bernoulli_samples)

Help on function cal_n in module calculate_n:

cal_n(p, exp_diff, conf_level=0.95, type='ave_yn', data=None, max_iter=100)
    calculate N based on:
    1. expected rate: `p` of which values are in [0,1]
    2. expected confidence interval: `exp_diff`
    3. confidence level: `conf_level`
    4. type of the statistics used, which may be one of the following:
        - 'ave_yn' is for when the statistic is the average of a bernoulli random variable, e.g. when you want to calc the average of clicks (since click event is a 0-1 random variable.) For this type, the standard deviation calculation will use the formula of Bernoulli's distribution.
        - 'ave_others' is when the statistic is the average of an unknown distribution. We will use a monte-carlo method to estimate sd.

Help on function cal_ci in module calculate_n:

cal_ci(data, conf_level=0.95)
    cal confidence interval (ci) assuming that the statistic is normally distributed, e.g. mean.

Help on function cal_n_from_t_test in 

# Examples

## Example 1

In [3]:
"""
Example 1: find N needed to do A/B testing based on delinquency rate

Since the dlq rate can be thought of as an average of a bernoulli rv as follows. Let x be a rv that can take the values 0 or 1, representing good or bad, respectively.

Let's say that typically the dlq rate of this group as a whole is around 5%
We expect that with a new strategy, the dlq rate should decrease to about 3%
Let's assume we want to consider the 95% confidence interval
"""

expected_base_dlq_rate = 0.02
expected_diff = 0.01
confidence_level = 0.88

num_samples = cal_n(p=expected_base_dlq_rate, exp_diff=expected_diff, conf_level=confidence_level, type='ave_yn')
print(num_samples)

474.0


In [11]:
num_samples = 70
p = 0.015
sd = np.sqrt(p*(1-p))
z = _get_z(0.90)
ci = z*sd/np.sqrt(num_samples)
print('The mean is: ' + str(round(p*100, 2)) + ' +/- '+str(round(ci*100, 2)))

The mean is: 1.5 +/- 2.39


In [5]:
"""
Once we have the samples, we can calculate the observed confidence interval here.
"""
# data = np.random.choice([0,1],size=num_samples, p =[1-p, p] ) #random data for an example.
#supposed we have around 0.047-0.048 dlq rate
p = 0.02
data = create_bernoulli_samples(p, num_samples)

mu = np.mean(data)
ci = cal_ci(data, conf_level=0.9)
print('The mean is: ' + str(mu*100) + ' +/- '+str(ci*100))

The mean is: 1.4285714285714286 +/- 2.3329462992860104


## Example 2

In [6]:
"""
Example 2: find N needed to do A/B testing based on delinquency rate. This is the same as in Example 1, but we will use the t-test method instead.

Let's say that typically the dlq rate of this group as a whole is around 8%
We expect that with a new strategy, the dlq rate should decrease to about 5%
Let's assume we want to consider the 95% confidence interval
"""
expected_base_dlq_rate = 0.05
expected_diff = 0.03
confidence_level = 0.95

num_samples = cal_n_from_t_test(p=expected_base_dlq_rate, exp_diff=expected_diff, conf_level=confidence_level, type='ave_yn')
print(num_samples)

146.0


In [7]:
"""
Once we have the samples, we can check the t-test and ci here.
"""
mu = 0.021
base_mean = 0.05
sd = mu*(1-mu)
t = (mu - base_mean) / (sd / np.sqrt(num_samples))
t_lookup = stats.t.ppf(q=confidence_level, df=num_samples-1)
print(np.abs(t))
print(t_lookup)
if np.abs(t) > t_lookup:
    print('Since t-statistic is greater than the 0.05 t-value, we can reject the null hypothesis that the mean is 8%.')

data = create_bernoulli_samples(mu, num_samples)
mu = np.mean(data)
ci = cal_ci(data, conf_level=confidence_level)
print('The mean is: ' + str(mu*100) + ' +/- '+str(ci*100))

17.04403585944076
1.6554302514146266
Since t-statistic is greater than the 0.05 t-value, we can reject the null hypothesis that the mean is 8%.
The mean is: 2.054794520547945 +/- 2.3011633760526182


## Example 3

In [8]:
"""
Example 3: Given a total samples of n, we want to split n into two groups to perform A/B testing. What would be the ratio of n1 : n2?

To do this, we use the two-sample t-test by providing the expected averages of the two groups. In this example, we assume that the rv is bernoulli.
"""

total_samples = 2000
expected_ave_1 = 0.05
expected_ave_2 = 0.07
min_ratio, max_ratio  = cal_ratio_for_comparing_two_means(total_n=total_samples, exp_mean_1=expected_ave_1, exp_mean_2=expected_ave_2, conf_level=0.95, type='ave_yn')

current total_n = 2000
0.23 < sample ratio (r) < 0.71,
where n1 = r*total_n and n2 = (1-r)*total_n


## Example 4

In [9]:
"""
Example 4: Given a total samples of n, we want to split n into two groups to perform A/B testing. What would be the ratio of n1 : n2?

To do this, we use the two-sample t-test by providing the expected averages of the two groups. In this example, we input expected mean and sd into the function directly.
"""

total_samples = 200
expected_ave_1 = 0.05
expected_ave_2 = 0.07
expected_sd_1 = 0.1
expected_sd_2 = 0.15

min_ratio, max_ratio  = cal_ratio_for_comparing_two_means(total_n=total_samples, exp_mean_1=expected_ave_1, exp_mean_2=expected_ave_2, conf_level=0.95, type='ave_others', sd_1 = expected_sd_1, sd_2 = expected_ave_2)

current total_n = 200
0.56 < sample ratio (r) < 0.62,
where n1 = r*total_n and n2 = (1-r)*total_n
